In [ ]:
## What it does: generates read .bed files separating reads that have a given block exon
#               either skipped or included.
## Date modified: 2025/04/16
## Inputs: gtf, gene_name, block_exons_ddPSImax0.2_adjacent.csv (output from script 3), .bam file from 
#         alignment (long-reads aligned e.g. with minimap2).

In [12]:
#import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pybedtools
from tqdm import tqdm
import seaborn as sns

In [13]:
#convert gtf to gene BED
def gtf_to_gene_bed(gtf_file, output_bed):
    """
    Convert a GTF file to a BED file with gene-level annotations.
    Ensures BED is properly formatted with integer coordinates and tab-delimited.
    """
    gtf = pd.read_csv(gtf_file, sep="\t", comment="#", header=None, names=[
        'chrom', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute'])

    # Filter only genes
    genes = gtf[gtf['feature'] == 'gene'].copy()

    # Extract gene_name from attributes
    genes['gene_name'] = genes['attribute'].str.extract('gene_name "([^"]+)"')

    # BED format requires 0-based start coordinates
    genes['start'] = genes['start'] - 1

    # Force proper formatting
    bed = genes[['chrom', 'start', 'end', 'gene_name', 'strand']].copy()
    bed.insert(4, 'score', 0)  # Insert score in column 5

    # Ensure integer coords
    bed['start'] = bed['start'].astype(int)
    bed['end'] = bed['end'].astype(int)

    # Save as TAB-separated BED file
    bed.to_csv(output_bed, sep="\t", header=False, index=False)

    print(f"BED saved with {len(bed)} gene entries to: {output_bed}")


In [14]:
# generate gene.bed from GTF
gtf_to_gene_bed("gencode.v44.basic.annotation.gtf", "hg38_genes.bed")

BED saved with 62700 gene entries to: hg38_genes.bed


In [15]:
# block definition
def get_exon_blocks(df, gene_name, max_dpsi_diff=0.2):
    """
    Group exons into blocks by dPSI similarity and adjacency.
    Returns a list of DataFrames, each one being a block.
    """
    df = df[df['gene_name'] == gene_name].sort_values('exon_number').reset_index(drop=True)
    df['group_id'] = ((df['dPSI'].diff().abs() > max_dpsi_diff) | 
                      (df['exon_number'].diff() != 1)).cumsum()

    blocks = [group for _, group in df.groupby('group_id')]
    return blocks


In [16]:
#Intersect reads with all exons in the selected block

def split_reads_for_block(gene_bed_path, gene_name, reads_bam_path, coord_exon_file, block_index=0, mode="full-block"):
    """
    Split reads into spliced-in or spliced-out for a single exon block.

    Parameters:
    - gene_bed_path: BED file with gene coordinates
    - gene_name: name of the gene to process
    - reads_bam_path: BAM file with aligned reads
    - coord_exon_file: CSV with exon block coordinates (e.g., block_exons_ddPSImax0.2_adjacent.csv)
    - block_index: which block to use (0 = first block found in gene)
    - mode: 'full-block' (default) or 'single-exon' 

    Returns:
    - spliced_in, spliced_out as BedTool objects
    """

    # Load gene BED and filter to gene
    gene_bed_df = pd.read_csv(gene_bed_path, sep="\t", header=None,
                              names=["chrom", "start", "end", "gene_name", "score", "strand"])
    gene_row = gene_bed_df[gene_bed_df['gene_name'] == gene_name]
    if gene_row.empty:
        raise ValueError(f"Gene {gene_name} not found in gene BED.")
    gene_bed = pybedtools.BedTool.from_dataframe(gene_row)

    # Read BAM, convert to BED12, clean chrom
    bam_df = pybedtools.BedTool(reads_bam_path).bamtobed(bed12=True).to_dataframe()
    bam_df['chrom'] = bam_df['chrom'].apply(lambda x: f"chr{x}" if not str(x).startswith("chr") else x)
    reads_bed = pybedtools.BedTool.from_dataframe(bam_df)

    # Get reads on the gene
    reads_on_gene = reads_bed.intersect(gene_bed, s=True, f=0.9)

    # Load exon table and get blocks
    blocks_df = pd.read_csv(coord_exon_file)
    exon_blocks = get_exon_blocks(blocks_df, gene_name)
    if block_index >= len(exon_blocks):
        raise IndexError(f"Block index {block_index} is out of range for {gene_name}")

    block_df = exon_blocks[block_index]
    strand = block_df['Strand'].iloc[0]

    # Convert block exons to BED format
    coords = block_df['Coord'].str.extract(r'(chr[\w\d]+):(\d+)-(\d+)')
    coords.columns = ['chrom', 'start', 'end']
    coords['name'] = gene_name
    coords['score'] = 0
    coords['strand'] = strand
    coords = coords.astype({'start': int, 'end': int})
    block_bed = pybedtools.BedTool.from_dataframe(coords)

    # Intersect logic
    if mode == "full-block":
        spliced_in = reads_on_gene
        for exon in block_bed:
            exon_bt = pybedtools.BedTool(str(exon), from_string=True)
            spliced_in = spliced_in.intersect(exon_bt, s=True, u=True)
        spliced_out = reads_on_gene.filter(lambda r: r.name not in {x.name for x in spliced_in}).saveas()

    elif mode == "single-exon":
        first_exon = block_bed[0]
        exon_bt = pybedtools.BedTool(str(first_exon), from_string=True)
        spliced_in = reads_on_gene.intersect(exon_bt, s=True, split=True, wa=True)
        spliced_out = reads_on_gene.intersect(exon_bt, s=True, wa=True).intersect(exon_bt, s=True, split=True, wa=True, v=True)

    else:
        raise ValueError("mode must be either 'full-block' or 'single-exon'")

    print(f"[{gene_name} block {block_index}] Spliced-in reads: {len(spliced_in)}")
    print(f"[{gene_name} block {block_index}] Spliced-out reads: {len(spliced_out)}")

    return spliced_in, spliced_out


In [17]:
#save reads

def save_balanced_reads(splice_in, splice_out, file_in, file_out, read_number=50):
    percent_in = len(splice_in) / (len(splice_in) + len(splice_out) + 1e-10)
    splice_in.to_dataframe().head(int(read_number * round(percent_in, 1))).to_csv(file_in, sep="\t", index=False, header=False)
    splice_out.to_dataframe().head(int(read_number * round(1 - percent_in, 1))).to_csv(file_out, sep="\t", index=False, header=False)


In [20]:
# USAGE: 
spliced_in, spliced_out = split_reads_for_block(
    gene_bed_path="hg38_genes.bed",
    gene_name="FUS",
    reads_bam_path="sm_dano_3_Tsm.sam.bam",
    coord_exon_file="block_exons_ddPSImax0.2_adjacent.csv",
    block_index=0,
    mode="single-exon"  
)


/Users/alexandrabergfort/opt/anaconda3/envs/jupyter/lib/python3.9/site-packages/pybedtools/bedtool.py:3698: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  return pandas.read_csv(self.fn, *args, sep="\t", **kwargs)


[FUS block 0] Spliced-in reads: 321
[FUS block 0] Spliced-out reads: 46


In [19]:
save_balanced_reads(spliced_in, spliced_out,
                    file_in="FUS_block0_included.bed",
                    file_out="FUS_block0_skipped.bed",
                    read_number=50)